In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from datetime import datetime
import re
import string
import pickle
import requests
import json

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
model = pickle.load(open('./old/modelSVC.pickle','rb'))
tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
APIurl = 'http://127.0.0.1:3000'


/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_10883/1179595794.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  model = pickle.load(open('./old/modelSVC.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_10883/1179595794.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9

In [3]:
def scraperKey(keyword,date_since,date_until):
    query = keyword+" lang:id until:"+str(date_until)+" since:"+str(date_since)
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    requests.post('{}/progress/twitter/new'.format(APIurl),{'dateGet':now,'keyword':keyword,'dateSince':date_since,'dateUntil':date_until,'status':1,'source':'tw'} )
    #print(query)
    #print(datetime.now())
    #print("Sedang Mengumpulkan Data Twitter...")
    tweets = []
    while len(tweets) <= 20:
        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            tweets.append([datetime.now().date(), keyword, tweet.date, tweet.user.username, tweet.content, tweet.hashtags,tweet.mentionedUsers, tweet.likeCount, tweet.retweetCount, tweet.replyCount])
        
        
        
    df = pd.DataFrame(tweets, columns=['timestamp','keyword','date', 'user', 'tweet', 'hashtags', 'mentions', 'likeCount', 'retweetCount', 'replyCount'])
    return df,now


In [4]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = text.encode("ascii", "ignore").decode() #remove emojis

    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    text = [w for w in text if not w in listStopwords]
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    #factory = StemmerFactory()
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

alay_dict = pd.read_csv('colloquial-indonesian-lexicon.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

def normalize_alay(text):
 text = ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])
 text = re.sub(' +', ' ', text)
 return text



In [5]:
df, now = scraperKey('ibu kota nusantara','2022-12-13','2023-01-18')
topic = 'IKN'

/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_10883/3241416775.py:11: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([datetime.now().date(), keyword, tweet.date, tweet.user.username, tweet.content, tweet.hashtags,tweet.mentionedUsers, tweet.likeCount, tweet.retweetCount, tweet.replyCount])


In [6]:
now

'2023-01-20 06:46:45'

In [7]:
id = requests.get('{}/progress/twitter/id/{}'.format(APIurl,now)).json()[0]['id']
print(id)


36


In [8]:
def update(code):
    requests.put('{}/progress/twitter/update'.format(APIurl), data={'id':id,'status': code})

In [9]:

if df.empty:
    update(404)
    print("Tidak ada data atau query keliru!")
else:
    df = df.drop_duplicates(subset=['tweet'])
    dataset = df
    update(2)
    #print(dataset)
    print("Sedang Membersihkan Data Twitter...")
    #gabungin hashtags
    # dataset['Hashtags'] = '-'.join(dataset['Hashtags'])
    print(datetime.now())
    print('cleaning text')
    dataset['text_clean'] = dataset['tweet'].apply(cleaningText)
    print(datetime.now())
    print('casefolding text')
    dataset['text_clean'] = dataset['text_clean'].apply(casefoldingText)
    dataset['text_clean'] = dataset['text_clean'].apply(normalize_alay)
    
    print(datetime.now())
    print('tokenizing text')
    dataset['text_preprocessed'] = dataset['text_clean'].apply(tokenizingText)
    print(datetime.now())
    print("filtering text")
    dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(filteringText)
    print(datetime.now())
    print('stemming text')
    dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(stemmingText)
    dataset["popularityScore"] = (dataset["likeCount"] + dataset["retweetCount"] + dataset["replyCount"])/3
    print("Selesai Membersihkan Data Twitter :)")
    print(datetime.now())

        ###################################
        ###      SENTIMENT ANALYSIS     ###
        ###################################

    print("Sedang Menganalisis Sentimen Publik...")
    update(3)

    print(datetime.now())
    # Make text preprocessed (tokenized) to untokenized with toSentence Function
    X = dataset['text_preprocessed'].apply(toSentence)
    X = tfidf_vectorizer.transform(X.values)
    
    # e = X.toarray()
    # #rumus mean
    # n = 0
    # mean = []
    # for i in e:
    #     a = sum(i.tolist())/5000
    #     mean.append(a)

    # #standar deviasi
    # n = 0
    # stdev = []
    # for i in e:
    #     s = np.std(i.tolist())
    #     stdev.append(s)

    # #max value
    # n = 0
    # maks = []
    # for i in e:
    #     a = max(i.tolist())
    #     maks.append(a)

    # #sum
    # n = 0
    # summ = []
    # for i in e:
    #     a = sum(i.tolist())
    #     summ.append(a)

    # #count feature
    # n = 0
    # count = []
    # for i in e:
    #     a = sum(map(lambda x : x != 0, i.tolist()))
    #     count.append(a)

    # extra_X = np.column_stack((e, np.array(mean), np.array(stdev), np.array(maks), np.array(summ), np.array(count)))

    # from scipy import sparse
    # extra_X = sparse.csr_matrix(extra_X)

    y_pred = model.predict(X)
    dataset['sentiment'] = y_pred

    polarity_decode = {0 : 'Negative', 1 : 'Neutral', 2 : 'Positive'}
    dataset['sentiment'] = dataset['sentiment'].map(polarity_decode)
    print("Selesai Menganalisis Sentimen Publik :)")

    print(datetime.now())

    dataset['date'] = pd.to_datetime(dataset['date']).dt.date
    dataset['date']=dataset['date'].astype(str)
        
    print(dataset.dtypes)

    

Sedang Membersihkan Data Twitter...
2023-01-20 06:48:25.819112
cleaning text
2023-01-20 06:48:25.831474
casefolding text
2023-01-20 06:48:25.840418
tokenizing text
2023-01-20 06:48:25.941240
filtering text
2023-01-20 06:48:26.179068
stemming text
Selesai Membersihkan Data Twitter :)
2023-01-20 06:48:26.372214
Sedang Menganalisis Sentimen Publik...
2023-01-20 06:48:26.374111
Selesai Menganalisis Sentimen Publik :)
2023-01-20 06:48:26.878446
timestamp             object
keyword               object
date                  object
user                  object
tweet                 object
hashtags              object
mentions              object
likeCount              int64
retweetCount           int64
replyCount             int64
text_clean            object
text_preprocessed     object
popularityScore      float64
sentiment             object
dtype: object


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1775 entries, 0 to 1805
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          1775 non-null   object 
 1   keyword            1775 non-null   object 
 2   date               1775 non-null   object 
 3   user               1775 non-null   object 
 4   tweet              1775 non-null   object 
 5   hashtags           1214 non-null   object 
 6   mentions           155 non-null    object 
 7   likeCount          1775 non-null   int64  
 8   retweetCount       1775 non-null   int64  
 9   replyCount         1775 non-null   int64  
 10  text_clean         1775 non-null   object 
 11  text_preprocessed  1775 non-null   object 
 12  popularityScore    1775 non-null   float64
 13  sentiment          1775 non-null   object 
dtypes: float64(1), int64(3), object(10)
memory usage: 208.0+ KB


In [11]:
df = dataset[['timestamp', 'keyword', 'date', 'user', 'tweet', 'hashtags', 'mentions', 'likeCount', 'retweetCount', 'replyCount','popularityScore',  'sentiment']]
df

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount,popularityScore,sentiment
0,2023-01-20,ibu kota nusantara,2023-01-17,Irakoestomo,• Arah kebijakan ketujuh adalah percepatan pem...,"[BappenasAdalahKita, suharsomonoarfa]",[https://twitter.com/Suharso_M],2,1,1,1.333333,Neutral
1,2023-01-20,ibu kota nusantara,2023-01-17,KerjaHasil_id,"Audiensi dengan Presiden, Komunitas Melayu-Ban...",None,None,12,2,1,5.000000,Positive
2,2023-01-20,ibu kota nusantara,2023-01-17,detikfinance,Kementerian Pekerjaan Umum dan Perumahan Rakya...,None,None,0,0,0,0.000000,Positive
3,2023-01-20,ibu kota nusantara,2023-01-17,Hadimakpul,"""Dalam pertimbangan kami, kajian kami bersama,...",None,None,0,0,1,0.333333,Positive
4,2023-01-20,ibu kota nusantara,2023-01-17,Hadimakpul,Pembangunan Ibu Kota Nusantara (IKN) mendapat ...,None,[https://twitter.com/jokowi],0,0,1,0.333333,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2023-01-20,ibu kota nusantara,2022-12-13,ginayaginasay,Pemerintah minta pelaku ekonomi kreatif kukar ...,"[WujudkanRIMaju, EkonomiPulih]",None,0,0,0,0.000000,Positive
1802,2023-01-20,ibu kota nusantara,2022-12-13,Bajingur5,Adanya Ibu Kota Nusantara (IKN) membuka peluan...,"[WujudkanRIMaju, EkonomiPulih]",None,1,0,0,0.333333,Neutral
1803,2023-01-20,ibu kota nusantara,2022-12-13,AdenGanteng00,Pemerintah mulai membangun sebanyak 36 rumah d...,"[WujudkanRIMaju, WajibboosterDisiplinProkes]",None,0,0,0,0.000000,Positive
1804,2023-01-20,ibu kota nusantara,2022-12-13,JPanpaan,Pemerintah saat ini sudah mulai untuk membangu...,"[WujudkanRIMaju, WajibboosterDisiplinProkes]",None,0,0,0,0.000000,Positive


In [12]:
df

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount,popularityScore,sentiment
0,2023-01-20,ibu kota nusantara,2023-01-17,Irakoestomo,• Arah kebijakan ketujuh adalah percepatan pem...,"[BappenasAdalahKita, suharsomonoarfa]",[https://twitter.com/Suharso_M],2,1,1,1.333333,Neutral
1,2023-01-20,ibu kota nusantara,2023-01-17,KerjaHasil_id,"Audiensi dengan Presiden, Komunitas Melayu-Ban...",None,None,12,2,1,5.000000,Positive
2,2023-01-20,ibu kota nusantara,2023-01-17,detikfinance,Kementerian Pekerjaan Umum dan Perumahan Rakya...,None,None,0,0,0,0.000000,Positive
3,2023-01-20,ibu kota nusantara,2023-01-17,Hadimakpul,"""Dalam pertimbangan kami, kajian kami bersama,...",None,None,0,0,1,0.333333,Positive
4,2023-01-20,ibu kota nusantara,2023-01-17,Hadimakpul,Pembangunan Ibu Kota Nusantara (IKN) mendapat ...,None,[https://twitter.com/jokowi],0,0,1,0.333333,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2023-01-20,ibu kota nusantara,2022-12-13,ginayaginasay,Pemerintah minta pelaku ekonomi kreatif kukar ...,"[WujudkanRIMaju, EkonomiPulih]",None,0,0,0,0.000000,Positive
1802,2023-01-20,ibu kota nusantara,2022-12-13,Bajingur5,Adanya Ibu Kota Nusantara (IKN) membuka peluan...,"[WujudkanRIMaju, EkonomiPulih]",None,1,0,0,0.333333,Neutral
1803,2023-01-20,ibu kota nusantara,2022-12-13,AdenGanteng00,Pemerintah mulai membangun sebanyak 36 rumah d...,"[WujudkanRIMaju, WajibboosterDisiplinProkes]",None,0,0,0,0.000000,Positive
1804,2023-01-20,ibu kota nusantara,2022-12-13,JPanpaan,Pemerintah saat ini sudah mulai untuk membangu...,"[WujudkanRIMaju, WajibboosterDisiplinProkes]",None,0,0,0,0.000000,Positive


In [13]:
# df[['timestamp','date']] = df[['timestamp','date']].astype(str)

In [14]:
# api = df.to_json("./test_data/json.json",orient='records')
data_dict = df.to_dict(orient='records')

In [15]:


for i in data_dict:
    dicts ={
        'dateGet':i['timestamp'],
        'keyword':i['keyword'],
        'contentDate':i['date'],
        'username':i['user'],
        'tweet':i['tweet'],
        'hashtags':i['hashtags'],
        'mentions':i['mentions'],
        'likeCount':i['likeCount'],
        'retweetCount':i['retweetCount'],
        'replyCount':i['replyCount'],
        'popularityScore':i['popularityScore'],
        'sentiment':i['sentiment'],
        'topic':topic
        }
    # print(dicts)
    c = requests.post('{}/data/twitter'.format(APIurl),dicts )
    print(c)

update(4)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [500]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [16]:
# cleanData[['Tweet', 'Result Prediction', 'text_preprocessed']].to_csv('pemilu.csv')
  


In [17]:
df_hashtags = df[['keyword','date','hashtags','sentiment']]
df_hashtags = df_hashtags[df_hashtags['hashtags'].notna()].reset_index()
df_mentions = df[['keyword','date','mentions','sentiment']]
df_mentions = df_mentions[df_mentions['mentions'].notna()].reset_index()

In [18]:
df_mentions = df[['keyword','date','mentions','sentiment']]
df_mentions = df_mentions[df_mentions['mentions'].notna()].reset_index()

In [19]:
df_mentions

,index,keyword,date,mentions,sentiment
0,0,ibu kota nusantara,2023-01-17,[https://twitter.com/Suharso_M],Neutral
1,4,ibu kota nusantara,2023-01-17,[https://twitter.com/jokowi],Positive
2,14,ibu kota nusantara,2023-01-17,[https://twitter.com/PadmavatiPrati1],Positive
3,39,ibu kota nusantara,2023-01-17,[https://twitter.com/walhinasional],Positive
4,40,ibu kota nusantara,2023-01-17,[https://twitter.com/ikn_id],Positive
...,...,...,...,...,...
150,1763,ibu kota nusantara,2022-12-14,[https://twitter.com/Indrafirman7777],Neutral
151,1788,ibu kota nusantara,2022-12-13,[https://twitter.com/detikcom],Positive
152,1792,ibu kota nusantara,2022-12-13,"[https://twitter.com/kemkominfo, https://twitt...",Positive
153,1795,ibu kota nusantara,2022-12-13,"[https://twitter.com/BKIPMBalikpapan, https://...",Neutral


In [20]:
for i in range(df_hashtags.shape[0]):
    for j in range(len(df_hashtags['hashtags'][i])):
        contentDate = df_hashtags['date'][i]
        keyword = df_hashtags['keyword'][i]
        hashtag = df_hashtags['hashtags'][i][j]
        sentiment = df_hashtags['sentiment'][i]
        if hashtag != 0:
            requests.post('{}/data/twitter/content'.format(APIurl),{
                'contentDate' : contentDate,
                'topic' : topic,
                'keyword' : keyword,
                'content' : hashtag,
                'type' : 'hashtag',
                'sentiment' : sentiment
            } )
    
    

In [21]:
for i in range(df_mentions.shape[0]):
    for j in range(len(df_mentions['mentions'][i])):
        contentDate = df_mentions['date'][i]
        keyword = df_mentions['keyword'][i]
        mention = str(df_mentions['mentions'][i][j]).replace('https://twitter.com/', '@')
        sentiment = df_mentions['sentiment'][i]
        if mention != 0:
            requests.post('{}/data/twitter/content'.format(APIurl),{
                'contentDate' : contentDate,
                'topic' : topic,
                'keyword' : keyword,
                'content' : mention,
                'type' : 'mention',
                'sentiment' : sentiment
            } )
            print(mention)
            
    
    

@Suharso_M
@jokowi
@PadmavatiPrati1
@walhinasional
@ikn_id
@atr_bpn
@ikn_id
@kementerian
@atr_bpn
@jokowi
@kementerian
@ruhutsitompul
@antontanjaya15
@eijkball
@ikn_id
@VegaLiana2
@HelmiFelis_
@jokowi
@kementerian
@jokowi
@atr_bpn
@kementerian
@kementerian
@fadyanova
@jokowi
@atr_bpn
@atr_bpn
@kementerian
@let
@kementerian
@let
@rakyatpembaruan
@ikn_id
@kementerian
@kementerian
@kementerian
@atr_bpn
@DitjenTataRuang
@atr_bpn
@BBCIndonesia
@kementerian
@kementerian
@ikn_id
@abangbelneg
@abangbelneg
@atr_bpn
@Nita_Mey_46
@kementerian
@kementerian
@let
@irfan_nuruddin
@kementerian
@Kemenkumham_RI
@atr_bpn
@democrazymedia
@KemenPU
@jokowi
@ElvinaNara
@BosPurwa
@YouTube
@IKNnusantaraID
@LawTop20
@atr_bpn
@geandin17
@sipalingyadhi
@jokowi
@nusantara_one
@IwanPangka
@PinkyStevanie
@jokowi
@aniesbaswedan
@NasDem
@KotaNusantara
@tukangnge
@jokowi
@setkabgoid
@jokowi
@jokowi
@jokowi
@anwaribrahim
@jokowi
@jokowi
@kompascom
@setkabgoid
@jokowi
@anwaribrahim
@setkabgoid
@jokowi
@anwaribrahim
@joko

In [22]:

# SELECT  content,
# 	COUNT( IF( sentiment = 'Positive',1,  NULL) ) AS 'positive',
# 	COUNT( IF( sentiment = 'Neutral',1, NULL) ) AS 'neutral',
# 	COUNT( IF( sentiment = 'Negative',1, Null) ) AS 'negative',
# 	COUNT(sentiment) AS 'total'
# FROM twitter_content
# WHERE type = 'mention'
# GROUP BY content
# ORDER BY total desc;